In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import hub
from langchain.document_loaders import Docx2txtLoader
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # 특정 경고 유형만 무시

#  3. DOCX 파일 로드 및 텍스트 추출 (Docx2txtLoader 활용)
def load_docx(file_path):
    """DOCX 파일에서 텍스트를 추출하는 함수."""
    try:
        loader = Docx2txtLoader(file_path)
        documents = loader.load()
        text = "\n".join([doc.page_content for doc in documents])
        if not text.strip():
            raise ValueError("문서에서 텍스트를 추출할 수 없습니다.")
        return text
    except Exception as e:
        raise RuntimeError(f"문서 로딩 실패: {str(e)}")

#  4. 문서 분할 함수
def split_text(text, chunk_size=500, chunk_overlap=50):
    """텍스트를 지정된 크기의 청크로 분할하는 함수."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return splitter.split_text(text)

#  5. 벡터 데이터베이스(FAISS) 생성 함수
def create_vector_store(text_chunks, embedding_model):
    """텍스트 청크를 임베딩하고 FAISS 벡터 저장소에 저장."""
    try:
        documents = [Document(page_content=chunk) for chunk in text_chunks]
        vector_store = FAISS.from_documents(documents, embedding_model)
        return vector_store
    except Exception as e:
        raise RuntimeError(f"벡터 저장소 생성 실패: {str(e)}")

#  6. LLM을 활용한 질문 응답 함수
def query_with_llm(query, vector_store):
    """LLM을 사용하여 검색된 문서 기반으로 답변 생성."""
    try:
        # LLM 모델 설정
        llm = ChatOpenAI(model_name="gpt-3.5-turbo")
        
        # 프롬프트 로드 (RAG 최적화된 LangChain Hub 프롬프트 사용)
        prompt = hub.pull("rlm/rag-prompt")

        # RetrievalQA 체인 생성
        qa_chain = RetrievalQA.from_chain_type(
            llm, 
            retriever=vector_store.as_retriever(),
            chain_type_kwargs={"prompt": prompt}
        )

        # LLM 응답 생성
        ai_message = qa_chain.invoke({"query": query})
        print(ai_message)
        return ai_message["result"]

    except Exception as e:
        raise RuntimeError(f"LLM 응답 생성 실패: {str(e)}")

#  실행 예제
if __name__ == "__main__":
    # DOCX 파일 경로
    docx_path = "data/tax_with_table_short.docx"
    
    # 1. 문서 로드
    text = load_docx(docx_path)
    print("문서 로드 완료")
    
    # 2. 문서 분할
    text_chunks = split_text(text)
    print(f"문서 분할 완료: {len(text_chunks)}개 청크 생성")
    
    # 3. 임베딩 모델 초기화
    embedding_model = OpenAIEmbeddings()
    
    # 4. 벡터 저장소 생성
    vector_store = create_vector_store(text_chunks, embedding_model)
    print("벡터 저장소 생성 완료")
    
    # 5. 질의 실행
    query = "총수입금액 불산입에 대하여 설명해 주세요."
    results = query_with_llm(query, vector_store)
    
    # 6. AI 응답 출력
    print("\n AI의 답변:")
    print(results)

문서 로드 완료
문서 분할 완료: 296개 청크 생성
벡터 저장소 생성 완료
{'query': '총수입금액 불산입에 대하여 설명해 주세요.', 'result': '총수입금액 불산입은 거주자의 총수입금액에 한정하여 과세 기간 중 공제되는 항목을 의미합니다. 예를 들어, 국세환급가산금, 지방세환급가산금, 그리고 부가가치세의 매출세액은 총수입금액에 산입하지 않습니다. 이에 관한 구체적인 내용은 대통령령에 의해 규정되어 있습니다.'}

 AI의 답변:
총수입금액 불산입은 거주자의 총수입금액에 한정하여 과세 기간 중 공제되는 항목을 의미합니다. 예를 들어, 국세환급가산금, 지방세환급가산금, 그리고 부가가치세의 매출세액은 총수입금액에 산입하지 않습니다. 이에 관한 구체적인 내용은 대통령령에 의해 규정되어 있습니다.


### 개선된 Source